In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from dash import html, dcc
import dash
from dash.dependencies import Input
from dash.dependencies import  Output

In [ ]:
df = pd.read_csv('../SpaceX-Prediction-and-Analysis/data_set/spacex_launch_dash.csv',
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})
lunch_sites = df['Launch Site'].unique().tolist()
# Randomly sample 500 data points. Setting the random state to be 42 so that we get same result.
# data = automobile_sales_data.sample(n=500, random_state=42)

# Pie Chart Creation
# Create a dash application

app = dash.Dash(__name__)

# Get the layout of the application and adjust it.
# Create an outer division using html.Div and add title to the dashboard using html.H1 component
# Add description about the graph using HTML P (paragraph) component
# Finally, add graph component.
app.layout = html.Div(children=[html.H1('Space X lunach Records Dashboard',style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
                                  html.Div(['Select lunch site', dcc.Dropdown(id='site-dropdown', options=[{'label':'All Site', 'value':'All'},
                                                                                                                  
                                                                                                                 {'label':lunch_sites[0], 'value':lunch_sites[0]},
                                                                                                                  {'label':lunch_sites[1], 'value':lunch_sites[1]},
                                                                                                                   {'label':lunch_sites[2], 'value':lunch_sites[2]},
                                                                                                                   {'label':lunch_sites[3], 'value':lunch_sites[3]},
                                                                                                                   
                                                                                                                 ],  searchable=True,
                                            placeholder='Select a Launch Site here',  style={'width':'50%','padding':'3px','height':'50px', 'font-size':20}),
                                            
                                            ],
                                         style={'font-size':35},),
                                html.Br(),
                                html.Br(),
                                html.Div([  dcc.Graph(id='output-container',style={'width':'50%', 'display':'flex'}),
                                          
                                          html.H3('Payload range (KG)',style={'textAlign': 'left', 'color': '#503D36', 'font-size': 24}),
                                           dcc.RangeSlider(id='payload-slider',  min=0, max=df['Payload Mass (kg)'].max(), step=100,
                                                marks={0: '0',1000: '1000',
                                                2500: '2500', 2500: '2500', 5000: '5000', 10000: '10000', 7500: '7500', 9000: '9000'},
                                                value=[0,10000]
                                                
                                                
                                                
                                                ),
                                            dcc.Graph(id='scatter-container',style={'width':'50%', 'display':'flex'})   
                                         ]),
                               
                                        
                               
                        ],
                      
                      )
                                
                                




@app.callback(
  [Output(component_id='output-container', component_property='figure'), Output(component_id='scatter-container', component_property='figure')],
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id="payload-slider", component_property="value")]
    
)
def get_scatter_chart(entered_site, range):
 
 
   fig2 = get_pie_chart(entered_site)
   figure  = ''
   if entered_site == 'All':
      figure=px.scatter(df,y='class', x='Payload Mass (kg)', color='Booster Version Category',
                              title=f'Total success Launches by ALL Site',)
              
   elif entered_site != None:
       df['Payload Mass (kg)']  = df['Payload Mass (kg)'].astype(float)
       filtered_df = df[(df['Launch Site'] == entered_site) & ( df['Payload Mass (kg)']>= range[0]) & (df['Payload Mass (kg)']<= range[1])]
      
               
       figure=px.scatter(filtered_df,y='class', x='Payload Mass (kg)', color='Booster Version Category',
           
       title=f'Total success Launches by {entered_site}')

       
      
    
            
    
   return [fig2, figure]

def get_pie_chart(entered_site):
    
    figure = ''
    if entered_site == 'All':
      
        figure=px.pie(df, values='class', 
        names='Launch Site', 
        title='Total success Launches by Launch site ')
    else:
           
            filtered_df = df.groupby('Launch Site')['class'].value_counts().reset_index()
            figure=px.pie(filtered_df[filtered_df['Launch Site'] == entered_site], values='count', 
                            names='class', 
                            title=f'Total success Launches by {entered_site}')
            
            
             
    return figure
          
    
                                
                                
                                
                                
                     
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
# Run the application                   
if __name__ == '__main__':
    host = '127.0.0.1'  # Change to '0.0.0.0' to make the app accessible externally
    port = 8050

    app.run_server(host=host, port=port)

    # Print the URL
    print(f"Dash app is running at http://{host}:{port}/")

**Observation** <br>
1. The launch site with the highest successful launch rate is **KSC LC-39A**, and it has the largest successful launch payload capacity at 38,463.65 kg.<br>

2. **VAFB SLC-4E** has the highest successful launch rate in the payload range(s) with a capacity of 96,000 kg. Its booster version category is **B4**. On the other hand, the lowest payload capacity is at CCAFS SLC-40 with 362 kg.

